In [1]:
! pip install -q langchain transformers sentence_transformers llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.9/770.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.2/370.2 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.8 MB/s eta 0:00:00


In [2]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex, GPTVectorStoreIndex, PromptHelper
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor, ServiceContext
import torch
from langchain.llms.base import LLM
from transformers import pipeline
from llama_index import Document
import logging

In [3]:
greeting="Hello! Welcome to Sara, your personal tour and hangout guide. How can I assist you today? Hi there! I am Sara, your go-to chatbot for tour plans, hangout suggestions, and restaurants. Hey, How can I help you today? Hey, it is great to see you! I am Sara, ready to provide you with amazing tour plans, hangout ideas, and restaurant recommendations. How can I make your day better? Hello and welcome! I am Sara, your friendly chatbot. How can I assist you in planning your next adventure or finding the perfect hangout spot?"

In [4]:
questions1="What is your desired travel destination or the region you are interested in? When are you planning to travel? Do you have specific dates in mind? How long do you plan to stay at your destination? What is the purpose of your trip? (Vacation, business, special occasion, etc.) Are you traveling alone or with a group? If with a group, how many people are in your party? What is your preferred travel style? (Luxury, budget, adventure, relaxation, cultural immersion, etc.) Do you have any specific interests or activities you would like to pursue during your trip? (Historical sites, outdoor adventures, culinary experiences, shopping, etc.) What is your approximate budget for the trip, including accommodations, transportation, and activities? Do you have any dietary restrictions or preferences that we should consider when recommending restaurants or meals? Are there any specific accommodations or amenities you require?  Have you traveled to this destination before? If yes, what are some places you have already visited or experiences you have had? Do you prefer a more guided tour experience or do you prefer to explore independently? Are there any specific landmarks or attractions you would like to prioritize during your trip? Are there any accessibility requirements or considerations we should keep in mind while planning your trip? Is there any additional information or special requests you would like to share to help us tailor your tour plan? What is your preferred mode of transportation within the destination? (Car rental, public transportation, private transfers, etc.) Are there any specific landmarks, attractions, or activities that are must-see or must-do for you? Are there any specific cultural events, festivals, or seasonal attractions happening during your travel dates that you would like to experience? Do you have any specific preferences for the type of accommodations? (Hotel, guesthouse, bed and breakfast, etc. What is your preferred proximity to city centers or tourist areas for accommodations? Are there any health or medical considerations that we should be aware of during the trip? Are there any visa requirements or restrictions that we should consider for your destination? Are you interested in any particular local cuisines or food experiences? Do you have any specific hobbies or interests that you would like to incorporate into your travel experience? (Photography, wildlife spotting, water sports, etc.) Are there any age-specific considerations or requirements for the travelers? (Traveling with children, elderly family members, etc.) Do you have any specific preferences for the pace of the trip? (Relaxed, packed with activities, balanced, etc.) Are there any specific souvenirs or shopping items you would like to prioritize during your trip? Do you require any assistance with travel insurance, travel documentation, or other administrative matters? Is there any specific budget allocation you would like us to consider for different aspects of your trip (accommodations, activities, dining, etc.)? Are there any specific cultural or historical sites you would like to explore during your trip? Do you have any specific preferences for the climate or season of your destination? Are there any specific language considerations or requirements we should keep in mind for your destination? Are there any mobility restrictions or accessibility needs we should consider for your trip? Do you have any specific preferences for the size or ambiance of the accommodations? (Cozy, boutique, large resort, etc.) Are you interested in any specific local experiences or off-the-beaten-path attractions? Are there any specific safety concerns or considerations we should address for your destination? Do you have any specific preferences for the proximity of accommodations to nature or outdoor activities? Are there any special celebrations or milestones happening during your trip that you would like to commemorate? Are you interested in any guided tours, excursions, or day trips during your stay? Do you have any preferred methods of communication during the trip (e.g., mobile app, email, phone)? Are there any specific types of restaurants or dining experiences you would like to explore? (Fine dining, street food, local specialties, etc.) Do you have any specific preferences for the proximity of accommodations to shopping areas or markets? Are there any specific requirements or preferences for the amenities provided by the accommodations? (Swimming pool, spa, gym, etc.) Do you have any particular interest in eco-tourism or sustainable travel options? There are plenty of great options for hanging out tonight! Are you in the mood for a lively atmosphere? ."
questions2="What are your preferred types of tourist destinations? (Historical, natural, cultural, etc.) Are you looking for popular tourist spots or off-the-beaten-path locations? Are you interested in outdoor activities like hiking, wildlife spotting, or water sports? Do you have any specific landmarks or attractions in mind that you would like to visit? Are you interested in exploring local markets, shopping districts, or entertainment venues? Are you open to visiting multiple destinations or do you prefer to stay in one place? Are there any specific festivals or events happening during your travel dates that you'd like to experience? Do you have any accessibility requirements or preferences that should be considered when recommending tour places? Are you interested in guided tours or do you prefer self-guided exploration? How much time do you have available for sightseeing and exploring tour places? What type of accommodation do you prefer?  What is your preferred accommodation budget range per night? Are you looking for accommodations in the city center or a quieter location? Do you have any specific amenities or facilities you require, such as a pool, fitness center, or spa? Are you open to staying in unique accommodations like eco-lodges, boutique hotels, or historic inns? Are there any specific neighborhoods or areas where you'd like to stay? How many people will be staying in the accommodation? Do you have any preferences regarding the view, such as cityscape, ocean, or mountain views? Are you interested in staying near specific attractions or landmarks? Are there any specific accommodation chains or brands that you prefer? Do you have any dietary restrictions or food preferences that we should consider when recommending restaurants? Are you interested in trying local cuisine and traditional dishes? Do you have a preference for fine dining, casual dining, or street food experiences? Are you interested in culinary tours or cooking classes to learn about local cuisine? Are there any specific types of restaurants or cuisines that you'd like to explore? Are you open to recommendations for hidden gems or local foodie spots? Do you have any specific food allergies that we should be aware of? Are you interested in wine or craft beer tastings during your travels? Do you prefer restaurants with vegetarian or vegan options? Are there any specific food markets or food festivals that you'd like to visit during your trip? Are you interested in visiting natural landscapes like mountains, beaches, or forests? Do you have any specific historical periods or eras that intrigue you? Are you looking for destinations that offer unique cultural experiences or interactions with local communities? Are you interested in exploring UNESCO World Heritage Sites or protected areas? Do you have any preferences regarding the climate or season for your travel destination? Are you interested in adventure activities like zip-lining, bungee jumping, or hot air ballooning? Are you looking for destinations that offer opportunities for wildlife safaris or marine life encounters? Do you prefer destinations with a vibrant nightlife and entertainment options? Are you interested in visiting museums, art galleries, or theaters during your travels? Are there any specific religious or spiritual sites that you'd like to include in your itinerary? Do you prefer accommodations with a traditional or modern design aesthetic? Are you interested in accommodations that offer unique experiences like staying in a castle or a treehouse? Are you looking for accommodations with family-friendly facilities or children's activities? Do you prefer accommodations with a beachfront or waterfront location? Are you open to staying in accommodations that promote sustainability and eco-friendly practices? Are you interested in accommodations with spa services or wellness facilities? Do you have any specific accessibility requirements for your accommodations? Are you interested in accommodations with a view of natural landscapes or city skylines? Do you prefer accommodations with in-house dining options or nearby restaurants? Are you open to staying in accommodations that offer unique amenities like hot springs or private pools? Are you interested in exploring local street food markets or night markets? Do you have any preferences for specific cooking styles or culinary traditions? Are you interested in food and wine pairing experiences or vineyard tours? Do you prefer restaurants with outdoor seating or scenic views? Are you looking for dining experiences that offer live music or entertainment? Are you interested in trying traditional food specialties or regional dishes? Do you have any specific dietary requirements or preferences, such as gluten-free or organic options? Are you open to trying adventurous or exotic food items during your travels? Do you prefer restaurants that focus on farm-to-table or sustainable dining practices? Are you interested in food tours or guided culinary experiences in local neighborhoods? "

In [5]:
rides="How many people will be traveling in your group? Do you prefer private transportation or are you open to shared rides? What is your preferred mode of transportation? (Car, van, bus, train, etc.) Are you interested in hiring a local driver/guide for your transportation needs? Do you have any specific requirements for luggage space or storage? Are you looking for a round-trip transportation option or one-way transport? Do you have any preferences for the type or class of vehicle? Are you open to alternative transportation options such as bicycles or scooters? Do you require any special accommodations for accessibility or mobility purposes? Are you interested in transportation with Wi-Fi or other onboard amenities? Do you have any preferences for a specific brand or model of vehicle? Are you interested in transportation options that offer scenic routes or sightseeing stops along the way? Do you require child seats or booster seats for young passengers? Are you looking for transportation services that offer airport transfers? Do you have any preferences for the language proficiency of the driver or guide? Are you interested in transportation options that offer flexibility in terms of pick-up and drop-off locations? Are there any specific features or services you would like to have in your transportation, such as air conditioning or charging ports? Do you have any preferences for the type of fuel or environmental considerations for your transportation choice? Are you interested in transportation options that offer audio guides or commentary during the journey? Do you have any specific budget constraints for your transportation needs? Are you looking for transportation services that offer 24/7 availability? Do you have any preferences for the duration of the ride or travel time? Are you interested in transportation options that provide door-to-door service?  Do you require any additional services along with your transportation, such as luggage assistance or meet-and-greet at the airport? Are there any specific restrictions or regulations you need to consider for your transportation choice, such as pet-friendly options or smoking policies? Do you prefer transportation services that allow for easy changes or modifications to your booking? Are you interested in transportation options that offer special discounts or packages for group bookings? Do you have any preferences for the level of comfort or luxury in your transportation choice? Are you open to alternative transportation modes like ferries or cable cars for specific routes or destinations? Do you have any preferences for the payment method or currency for your transportation booking? Are you interested in transportation options that offer flexibility in terms of pick-up and drop-off times? Do you require any specific safety features or certifications for your transportation choice? Are you looking for transportation services that offer live tracking or real-time updates? Do you have any preferences for the availability of storage space for equipment or gear? Are you interested in transportation options that offer multi-lingual support or translation services? Do you require any assistance with border crossings or visa-related transportation arrangements? Are you open to transportation options that provide a driver but allow you to self-drive? Do you have any preferences for the level of privacy or exclusivity in your transportation choice? Are you interested in transportation services that offer flexible cancellation or refund policies? Do you require any specific add-on services such as airport lounge access or VIP treatment? Are you looking for transportation options that are environmentally friendly or promote sustainability? Do you have any preferences for the type of roads or routes taken during your transportation?"
famous_tour="The Pyramids of Giza in Egypt are ancient structures that have stood for thousands of years. These iconic pyramids, including the Great Pyramid of Khufu, fascinate visitors with their architectural grandeur and historical significance. The Acropolis in Athens, Greece, is a UNESCO World Heritage Site and a symbol of ancient Greek civilization. It is home to the iconic Parthenon, a majestic temple dedicated to the goddess Athena, offering panoramic views of the city. The Great Wall of China is a monumental feat of engineering and stretches thousands of miles across China's landscape. This UNESCO World Heritage Site attracts visitors who can hike along its sections and marvel at its historical significance. The Taj Mahal in Agra, India, is an architectural masterpiece and one of the New Seven Wonders of the World. This ivory-white marble mausoleum, built by Emperor Shah Jahan in memory of his wife, showcases intricate craftsmanship and breathtaking beauty. The Colosseum in Rome, Italy, is an ancient amphitheater that hosted gladiatorial contests and other spectacles in the Roman era. It stands as a symbol of the grandeur of the Roman Empire and remains an awe-inspiring sight for visitors. The Louvre Museum in Paris, France, is one of the world's largest and most visited art museums. It houses a vast collection of artistic treasures, including Leonardo da Vinci's Mona Lisa, the Venus de Milo, and the Winged Victory of Samothrace. The Metropolitan Museum of Art, located in New York City, is a renowned cultural institution featuring a diverse collection spanning thousands of years. It showcases art from various civilizations, from ancient Egypt to contemporary masterpieces. The Prado Museum in Madrid, Spain, is home to a remarkable collection of European art. It houses masterpieces by renowned artists such as Francisco Goya, Diego Velázquez, and El Greco, offering visitors a journey through art history. The Hermitage Museum in St. Petersburg, Russia, is one of the world's largest and oldest museums, housing a vast collection of art and cultural artifacts. It occupies the opulent Winter Palace and showcases works by artists like Rembrandt, Van Gogh, and Monet. The National Gallery in London, United Kingdom, is an art museum housing an extensive collection of European paintings. It features iconic works by artists such as Leonardo da Vinci, Vincent van Gogh, and J.M.W. Turner. Paris, known as the City of Light, is home to iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. It is renowned for its rich history, romantic ambiance, and world-class cuisine. Rome, the Eternal City, boasts ancient wonders like the Colosseum, Roman Forum, and the Vatican City. Visitors can explore its rich archaeological sites, indulge in authentic Italian cuisine, and admire masterpieces in the Vatican Museums. New York City, the Big Apple, is a vibrant metropolis famous for its towering skyscrapers, Times Square, Central Park, and the Statue of Liberty. It offers a diverse cultural scene, Broadway shows, renowned art museums, and a melting pot of cuisines. Tokyo, the bustling capital of Japan, blends tradition and modernity. Visitors can experience the serenity of temples like Senso-ji and the vibrant energy of districts like Shibuya and Shinjuku. Tokyo is also known for its culinary delights, from sushi to ramen. Sydney, Australia's largest city, captivates with its stunning harbor, Sydney Opera House, and Bondi Beach. Visitors can explore the Royal Botanic Garden, climb the Sydney Harbour Bridge, and enjoy the city's vibrant arts and music scene. The Grand Canyon in Arizona is a breathtaking natural wonder known for its awe-inspiring size and vibrant colors. Its vast expanse offers stunning vistas, hiking trails, and the opportunity to experience the grandeur of nature. The Great Barrier Reef in Australia is the largest coral reef system in the world. It teems with marine life and offers snorkeling and diving opportunities to witness the vibrant coral formations and encounter colorful fish and marine species. The Amazon Rainforest, spanning multiple countries in South America, is a biodiversity hotspot and home to countless plant and animal species. Explorers can embark on eco-tours to witness its lush landscapes, encounter wildlife, and learn about indigenous cultures. Niagara Falls, located on the border of the United States and Canada, is a majestic natural wonder. The thundering waterfalls attract visitors who can enjoy boat tours, observation decks, and scenic viewpoints to witness the impressive power of the falls. Mount Everest, the highest peak in the world, lures adventurous climbers to attempt its summit. Situated in the Himalayas, Everest is a challenging and awe-inspiring destination for mountaineers seeking to conquer its formidable heights."
historical_tours="The Great Wall of China: Stretching over 13,000 miles, the Great Wall of China is a remarkable testament to human engineering and ingenuity. Originally built to protect the Chinese empire from invasions, it is a UNESCO World Heritage Site and offers visitors breathtaking views of the surrounding landscapes. Machu Picchu: Nestled high in the Andes Mountains of Peru, Machu Picchu is an ancient Inca citadel that has captivated explorers and historians alike. This archaeological marvel, built in the 15th century, showcases impressive stone craftsmanship and offers visitors a glimpse into the mysterious world of the Inca civilization. Petra: Known as the Rose City, Petra is an extraordinary archaeological site in Jordan. Carved into pink sandstone cliffs, this ancient Nabatean city features elaborate facades, tombs, and temples, including the iconic Treasury. Visitors can embark on a journey through time as they explore the intricacies of this UNESCO World Heritage Site. Angkor Wat: Situated in Cambodia, Angkor Wat is a magnificent temple complex and the largest religious monument in the world. Built in the 12th century, it showcases Khmer architecture and is a symbol of national pride. Visitors can wander through intricately carved stone structures and experience the mystical atmosphere of this UNESCO World Heritage Site. The Colosseum: Located in the heart of Rome, Italy, the Colosseum is an iconic symbol of the Roman Empire. This ancient amphitheater once hosted grand spectacles, gladiatorial contests, and public events. Its imposing architecture and rich historical significance make it a must-visit site for history enthusiasts and tourists. The Parthenon: Perched atop the Acropolis in Athens, Greece, the Parthenon is a magnificent temple dedicated to the goddess Athena. Constructed in the 5th century BC, it is an enduring symbol of ancient Greek civilization and showcases exquisite Doric architecture. Visitors can marvel at its grandeur and enjoy panoramic views of Athens. Chichen Itza: In the heart of the Yucatan Peninsula in Mexico, Chichen Itza is an archaeological site representing the Maya civilization. It features stunning structures like the iconic El Castillo pyramid, Temple of the Warriors, and the Great Ball Court. Visitors can explore the remnants of this ancient city and learn about its rich history and astronomical significance. The Pyramids of Giza: Situated on the outskirts of Cairo, Egypt, the Pyramids of Giza are a testament to the grandeur of the ancient Egyptian civilization. The Great Pyramid, built as a tomb for Pharaoh Khufu, is the largest of the three pyramids and remains one of the Seven Wonders of the Ancient World. Visitors can marvel at these colossal structures and delve into the mysteries of ancient Egypt."

In [6]:
text_list = [greeting,rides,questions1,questions2,famous_tour,historical_tours]

documents = [Document(t) for t in text_list]

In [7]:
class customLLM(LLM):
    model_name = "google/flan-t5-large"
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]
 
    def _identifying_params(self):
        return {"name_of_model": self.model_name}

    def _llm_type(self):
        return "custom"

llm_predictor = LLMPredictor(llm=customLLM())

In [8]:
hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)

In [9]:
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

In [10]:
logging.getLogger().setLevel(logging.CRITICAL)

In [11]:
query_engine = index.as_query_engine()
response = query_engine.query("Hello Sara")
print(response)

Hello and welcome! I am Sara, your friendly chatbot. How can I assist you in planning your next adventure or finding the perfect hangout spot?


In [12]:
query_engine = index.as_query_engine()
response = query_engine.query("Hi")
print(response)

Hello! Welcome to Sara, your personal tour and hangout guide. How can I assist you today?


In [13]:
query_engine = index.as_query_engine()
response = query_engine.query("What would be good residence plan?")
print(response)

Token indices sequence length is longer than the specified maximum sequence length for this model (1143 > 512). Running this sequence through the model will result in indexing errors


a beachfront or waterfront location


In [14]:
query_engine = index.as_query_engine()
response = query_engine.query("recommend restaurants")
print(response)

Do you have any dietary restrictions or food preferences that we should consider when recommending restaurants


In [15]:
query_engine = index.as_query_engine()
response = query_engine.query("where should i go for good food place?")
print(response)

restaurants with outdoor seating or scenic views


In [16]:
query_engine = index.as_query_engine()
response = query_engine.query("which is best historical place to visit?")
print(response)

The Colosseum


In [17]:
query_engine = index.as_query_engine()
response = query_engine.query("tell me about Great Pyramid")
print(response)

The Great Pyramid, built as a tomb for Pharaoh Khufu, is the largest of the three pyramids and remains one of the Seven Wonders of the Ancient World.


In [18]:
query_engine = index.as_query_engine()
response = query_engine.query("Where can i hangout tonight?")
print(response)

There are plenty of great options for hanging out tonight! Are you in the mood for a lively atmosphere


In [19]:
query_engine = index.as_query_engine()
response = query_engine.query("which rides would be suitable?")
print(response)

Car, van, bus, train
